<h3>Задание 1. Найти опорный план методом мин. тарифа, общую стоимость перевозок</h3><br>Билет 58

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
np.set_printoptions(suppress=True)

In [2]:
A = np.array([100, 110, 40])
B = np.array([100, 50, 20, 30])
C = np.array([[12, 5, 11, 6],
              [11, 15, 21, 10],
              [13, 15, 15, 11]])

N = len(A)
M = len(B)

df = pd.DataFrame(C, columns=B, index=A)
df.columns.name = 'Ai \\ Bj'
df

Ai \ Bj,100,50,20,30
100,12,5,11,6
110,11,15,21,10
40,13,15,15,11


Допустимый план перевозки груза методом минимальной стоимости

In [3]:
P = np.zeros_like(C)
INF = 1_000_000_007
C_ = C.copy()
A_ = A.copy()
B_ = B.copy()
i = 0
j = 0
while not np.all(C_ == INF):
    i, j = np.unravel_index(C_.argmin(), C_.shape)
    P[i][j] = min(A_[i], B_[j])
    if A_[i] <= B_[j]:
        C_[i].fill(INF)
        B_[j] -= A_[i]
        A_[i] = 0
    else:
        C_[:, j].fill(INF)
        A_[i] -= B[j]
        B_[j] = 0    
if np.count_nonzero(P) == N + M - 1:
    print('Опорный план невырожденный')
else:
    print('Опорный план вырожденный')

F = 0
for i in range(N):
    for j in range(M):
        F += C[i][j] * P[i][j]
print('Полученный план перевозки груза:\n', P)
print('Значение целевой функции для этого опорного плана F =', F)

Опорный план вырожденный
Полученный план перевозки груза:
 [[  0  50  20  30]
 [100   0   0   0]
 [  0   0   0   0]]
Значение целевой функции для этого опорного плана F = 1750


Оптимальный план перевозки груза

In [4]:
A_ub = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
b_ub = A
b_eq = B
A_eq = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])

res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})

X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 145.0               
0.2301046258201     0.2301046258201     0.2301046258201     0.783390459773   0.2301046258201     428.9964070985      
0.05984632167826    0.05984632167825    0.05984632167825    0.768803924024   0.05984632167828    970.047433581       
0.009056379799842   0.009056379799845   0.009056379799845   0.8755218620065  0.009056379799847   1528.194040229      
0.0001558297302771  0.0001558297302773  0.0001558297302773  0.9832033798974  0.0001558297302764  1745.673329304      
8.009899099223e-09  8.009899166247e-09  8.009899203574e-09  0.9999486166139  8.009899371862e-09  1749.999776749      
Optimization terminated successfully.
         Current function value: 1749.999777 
         Iterations: 5


In [5]:
print('Оптимальный план перевозки груза X*:\n', X)
print('Общая стоимость перевозки:\nF_min = F(X*) =', F_min)

Оптимальный план перевозки груза X*:
 [[  0.  50.  20.  30.]
 [100.   0.   0.   0.]
 [  0.   0.   0.   0.]]
Общая стоимость перевозки:
F_min = F(X*) = 1750.0
